#  Setup: Display options

In [2]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [1]:

#  Used to report timings
#

from timeit import default_timer

print("--")



--


# Setup: Connectivity

In [3]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [4]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [20]:

#  DELETE ONE SPECIFIC GRAPH ONLY

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name = l_database.name).graphs_in_database():
      if ((l_database.name == DB_NAME) and (l_graph.name == GRAPH_NAME)):   
         l_handle = my_client.get_database(name = l_database.name).get_graph_by_id(id = l_graph.graph_id)
         l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "   GRAPH Version: ", l_graph.version, "   Graph Name: ", l_graph.name)

print("--")


GRAPH ID:  DfmNBHbvwBWi3PzzdrpaBEY7uQ69ofWFh3bwLURyDW1m    GRAPH Version:  10    Graph Name:  my_graph_BB
--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [21]:

#  CREATE A GRAPH

my_graph=my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 9oxuVL3PJqNwMpBuuDshgohrh4mmgyAwfmSoM1tPXCoH, 0>


In [18]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 8v5HziwAJ9NPrBpMvME2MLfmSzN2u2MMn8KXvwqWq1PT, 1>


In [ ]:
# ##################################################################

In [7]:

#  Now that we share servers, get a better list of databases and graphs ..
#

for l_each_d in my_client.databases():
   print("Database Name: %-28s   Id: %s" % (l_each_d.name, l_each_d.database_id))
      #
   for l_each_g in my_client.get_database(name=l_each_d.name).graphs_in_database(): 
      print("   Graph Name: %-28s      Version: %-8d   Id: %s" % (l_each_g.name, l_each_g.version, l_each_g.graph_id))


print(""  )
print("--")


Database Name: my_db                          Id: 21JDRLcScJPzXu2CsX8hSCCwpuvBdXJbkLKQ7CdcKxV9
   Graph Name: my_graph_BB                       Version: 2          Id: DfmNBHbvwBWi3PzzdrpaBEY7uQ69ofWFh3bwLURyDW1m
   Graph Name: my_graph                          Version: 0          Id: uNTHrMVDLBBBv4QMwk4DswkRkGNeo6QxWcJBQAcEsKC
Database Name: default                        Id: 2ps7XNc4f8KLRVXwutJzqQVE3bTUdVbHw357hkA6PM5z

--


#  Step 00: Import LDBC from CSV

In [ ]:

#  COPY FILES TO/FROM S3 ..
#
#     https://console.cloud.google.com/storage/browser/katana-demo-datasets/
# 
#     Example,
#        gsutil cp   ./30_NodeHeader.txt      gs://farrell-data-bucket/30_NodeHeader.txt
#


In [ ]:

#  When we need a token for writing ..
#

# import os
# 
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/04_MyKeys.json"
# 
# print("--")


In [11]:

#  Import data into graph using csv
#

l_time = default_timer()


#  l_node = "gs://katana-demo-datasets/csv-datasets/ldbc/headers/sf-0.003-bi-node_headers.txt"
#  l_edge = "gs://katana-demo-datasets/csv-datasets/ldbc/headers/sf-0.003-bi-edge_headers.txt"

l_node = "gs://katana-demo-datasets/csv-datasets/ldbc/headers/sf-100-bi-node_headers.txt"
l_edge = "gs://katana-demo-datasets/csv-datasets/ldbc/headers/sf-100-bi-edge_headers.txt"
   #
l_data = "gs://katana-demo-datasets/csv-datasets/ldbc"


#  I thought this method was deprecated ?
#

import_data.csv(
   my_graph,
      #
   input_node_path    = l_node,
   input_edge_path    = l_edge,
   input_dir          = l_data,
      #
   data_delimiter     = "|",
   schema_delimiter   = "|",
      #
   files_have_headers = True,
   # partition_policy = "random-oec"
   )


print("Time in seconds: %d" % (default_timer() - l_time))

print("--")


#  Sample output,
#
#  Time in seconds: 1738           #  That's 28 minutes
#


          0/? [?op/s]

Time in seconds: 1738
--


In [ ]:

#  Import data using RDG
#
#     This is LDBC 100
#


l_time = default_timer()

import_data.rdg(my_graph, "gs://katana-demo-datasets/rdg-datasets/dslg6.stpg0/ldbc_003_")
#  import_data.rdg(my_graph, "gs://katana-demo-datasets/rdg-datasets/dslg6.stpg0/ldbc_100_")

print("Time in seconds: %d" % (default_timer() - l_time))
      
print("--")


#  Sample output,
#
#  Time in seconds: 1164            #  That's 19 minutes
#


          0/? [?op/s]

In [19]:

l_time = default_timer()

display(my_graph.num_nodes())
display(my_graph.num_edges())

print("Time in seconds: %d" % (default_timer() - l_time))
      
print("--")
      
      
#  Sample output,
#
#  (CSV)
#     189263961
#     1297157447
#     Time in seconds: 267
#
#  (RDG)
#
#  Got this twice ,
#
#     WorkerError: graph-worker exited while executing operation GbTxBptGrZE1mYVeGL1Sgvnn1NGmZe1nPwZ49wdmCRZ6-5T3T62xDgBVmaUs4R (killed [origin_code: '3', extra_info: 'exit status 137: graph-worker exited with wait status 35072
#



          0/? [?op/s]

WorkerError: graph-worker exited while executing operation DrizfRmTcPmhvEP31LwbnKWa5rKqmco27geYAqxs8qgg-46mKfTY8J5eW5QpRW (killed [origin_code: '3', extra_info: 'exit status 137: graph-worker exited with wait status 35072
']